### Lecture 4: Homework

Today we gonna learn how to choose between ML models, based on data type. Your task would be to predict **the edibility of a mushroom** based on sample descriptions (binary classification problem)

The **tricky part here is that 95% of the features are of categorical type.**
<br>That's the one where we would **(usually)  prefer tree-based algorithms over linear methods**

Although this dataset was originally contributed to the UCI Machine Learning repository nearly 30 years ago, mushroom hunting (otherwise known as "shrooming") is enjoying new peaks in popularity. Learn which features spell certain death and which are most palatable in this dataset of mushroom characteristics. And how certain can your model be?

This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom drawn from The Audubon Society Field Guide to North American Mushrooms (1981). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like "leaflets three, let it be'' for Poisonous Oak and Ivy.

More information can be found [here](https://www.kaggle.com/uciml/mushroom-classification/data)

Please find below correspondent [google form](https://docs.google.com/forms/d/e/1FAIpQLScmKfUApMlcD81u9UZxM7xG3vJiEJHrPrG-3b0i_jyPEDijgQ/viewform) to submit your answers

In [1]:
# library import
import pandas as pd
import numpy as np
from os.path import join as pjoin
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

# preprocessing / validation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
)
# ML models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# metrics
from sklearn.metrics import classification_report, f1_score

In [2]:
# read data
DATA_DIR = 'data'
df_train = pd.read_csv("/Users/prophet/Documents/GitHub/udscourse/data/4-mushrooms-train.csv", engine='c')
df_test = pd.read_csv("/Users/prophet/Documents/GitHub/udscourse/data/4-mushrooms-test.csv", engine='c')
print(df_train.shape, df_test.shape)

(6499, 23) (1625, 22)


In [3]:
# let's see what data looks like
df_train.head()

,target,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,0,convex,scaly,brown,bruises,pungent,free,close,narrow,white,enlarging,equal,smooth,smooth,white,white,partial,white,one,pendant,brown,scattered,urban
1,1,flat,fibrous,gray,bruises,none,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,white,partial,white,one,pendant,brown,several,woods
2,0,flat,smooth,brown,no,none,attached,close,broad,orange,enlarging,missing,smooth,smooth,orange,orange,partial,orange,one,pendant,brown,several,leaves
3,1,convex,fibrous,gray,bruises,none,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,white,partial,white,one,pendant,black,solitary,woods
4,0,knobbed,smooth,brown,no,foul,free,close,narrow,buff,tapering,missing,silky,smooth,pink,pink,partial,white,one,evanescent,white,several,paths


In [4]:
# for convenient calculations, let us merge train with test
df = pd.concat([df_train, df_test], axis=0)
# add column for filtering train/test
df['is_train'] = True
df.loc[df.target.isnull(), 'is_train'] = False
# check shapes
print(df.shape)
# check labels
df.is_train.value_counts()

(8124, 24)


True     6499
False    1625
Name: is_train, dtype: int64

### Task 1. Which feature has the highest amount of unique values? (joint dataset)


In [5]:
features_dict = {}

In [6]:
feature_list = list(df)
feature_list = feature_list[:-1]
feature_list.remove('target')

In [7]:
feature_list

['bruises',
 'cap_color',
 'cap_shape',
 'cap_surface',
 'gill_attachment',
 'gill_color',
 'gill_size',
 'gill_spacing',
 'habitat',
 'odor',
 'population',
 'ring_number',
 'ring_type',
 'spore_print_color',
 'stalk_color_above_ring',
 'stalk_color_below_ring',
 'stalk_root',
 'stalk_shape',
 'stalk_surface_above_ring',
 'stalk_surface_below_ring',
 'veil_color',
 'veil_type']

In [8]:
def get_unique_values(feature):
    
    u_values = set()
    
    for i in df[feature]:
        if i not in u_values:
            u_values.add(i)
            
    features_dict.update({feature: len(u_values)})
    print(feature, u_values)
    


In [9]:
for i in feature_list:
    get_unique_values(i)

bruises {'no', 'bruises'}
cap_color {'yellow', 'cinnamon', 'buff', 'pink', 'gray', 'red', 'green', 'brown', 'white', 'purple'}
cap_shape {'knobbed', 'sunken', 'flat', 'bell', 'conical', 'convex'}
cap_surface {'fibrous', 'smooth', 'scaly', 'grooves'}
gill_attachment {'free', 'attached'}
gill_color {'yellow', 'black', 'buff', 'orange', 'chocolate', 'pink', 'gray', 'red', 'green', 'brown', 'white', 'purple'}
gill_size {'broad', 'narrow'}
gill_spacing {'crowded', 'close'}
habitat {'leaves', 'woods', 'meadows', 'waste', 'urban', 'grasses', 'paths'}
odor {'musty', 'pungent', 'fishy', 'creosote', 'foul', 'anise', 'almond', 'spicy', 'none'}
population {'clustered', 'several', 'solitary', 'numerous', 'scattered', 'abundant'}
ring_number {'one', 'two', 'none'}
ring_type {'pendant', 'flaring', 'evanescent', 'large', 'none'}
spore_print_color {'yellow', 'black', 'chocolate', 'buff', 'orange', 'green', 'brown', 'white', 'purple'}
stalk_color_above_ring {'yellow', 'cinnamon', 'buff', 'orange', 'pink

In [10]:
print(features_dict)

{'bruises': 2, 'cap_color': 10, 'cap_shape': 6, 'cap_surface': 4, 'gill_attachment': 2, 'gill_color': 12, 'gill_size': 2, 'gill_spacing': 2, 'habitat': 7, 'odor': 9, 'population': 6, 'ring_number': 3, 'ring_type': 5, 'spore_print_color': 9, 'stalk_color_above_ring': 9, 'stalk_color_below_ring': 9, 'stalk_root': 5, 'stalk_shape': 2, 'stalk_surface_above_ring': 4, 'stalk_surface_below_ring': 4, 'veil_color': 4, 'veil_type': 1}


In [11]:
# your code goes here
# ---------------------------------------------------------------
most_diversive = 'gill_color'
# ---------------------------------------------------------------
print(most_diversive)

gill_color


### Task 2
**As a preparation, one would spend up to 15-30 minutes on exploratory data analysis (EDA)** - make sure you understand how features are distributed in train/test, what they look like, are they ordinal/binary/categorical before moving further
<br>While doing it, please answer the questions

#### 2.1 Are there any features, obviously redundant to train on? If yes - what are they and why it's better to remove them?

In [12]:
print(df.shape)
# your code/hardcoded list goes here
# ---------------------------------------------------------------
redundant_columns = [
    'veil_type'
]
# ---------------------------------------------------------------
# lets drop these columns from joint dataset
df = df.drop(redundant_columns, axis=1, errors='ignore')
print(df.shape)

(8124, 24)
(8124, 23)


####  2.2 How many features (excluding target variable and train/test indexing columns) are:
- categorical (more than 2 unique values, no explicit ordering)
- ordinal (more than 2 unique values, explicit ordering)
- binary (2 unique values, doesn't matter whether it has ordering or is "yes/no" styled) 

In [15]:
# your code goes here
# ---------------------------------------------------------------
ordinal_cols = sorted([
    'ring_number'
])
binary_cols = sorted([
    'bruises', 'gill_size', 'gill_attachment', 'gill_spacing', 'stalk_shape'
])
categorical_cols = sorted([
    'stalk_root', 'ring_type', 'odor', 'habitat', 'cap_surface', 'cap_shape', 'veil_color', 
    'spore_print_color', 'gill_color', 'cap_color', 'population', 'stalk_surface_above_ring', 'stalk_surface_below_ring',
    'stalk_color_above_ring', 'stalk_color_below_ring'
])
# ---------------------------------------------------------------
print('categorical: {}\nordinal: {}\nbinary: {}'.format(
    len(categorical_cols), len(ordinal_cols), len(binary_cols)))

categorical: 15
ordinal: 1
binary: 5


In [16]:
# To be used in training, data must be properly encoded
from collections import defaultdict

# function to encode categorical data


def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))


# label encode data (categorical + binary)
__encode_categorical(df_list=[df], cat_cols=categorical_cols+binary_cols)
# make sure you encode the only ordinal column in correct order
df[ordinal_cols[0]] = df[ordinal_cols[0]].map({'none': 0, 'one': 1, 'two': 2})

# define useful feature columns to be used for training
# (union of all columns discussed above)
columns_to_use = ordinal_cols + binary_cols + categorical_cols

### Task 3. Prepare cross-validation strategy and perform comparison of 2 baseline models (linear vs tree-based)

### =====================================================
#### Briefly about Validation / Cross-Validation

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but **would fail to predict anything useful on yet-unseen data. This situation is called overfitting**. 
<br>To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set ```X_test, y_test```. 
<br>Note that the word “experiment” is not intended to denote academic use only, because even in commercial settings machine learning usually starts out experimentally.

When evaluating different settings (“hyperparameters”) for estimators, **there is still a risk of overfitting on the test set** because the parameters can be tweaked until the estimator performs optimally. 
<br>This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. 
<br>To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, **we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.**

A solution to this problem is a procedure called **cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV**. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

- A model is trained using k-1 of the folds as training data;
- the resulting model is validated on the remaining part of the data 
<br>(i.e., it is used as a test set to compute a performance measure such as accuracy).
        
<img src="https://hsto.org/files/b1d/706/e6c/b1d706e6c9df49c297b6152878a2d03f.png"/ style="width:75%">

The performance measure reported by k-fold cross-validation **is then the average of the values computed in the loop**. 
<br>This approach can be computationally expensive, but does not waste too much data (as it is the case when fixing an arbitrary test set), which is a major advantage in problem such as inverse inference where the number of samples is very small.


Some classification problems can **exhibit a large imbalance in the distribution of the target classes: for instance there could be several times more negative samples than positive samples**. 
<br>In such cases it is recommended to use **stratified sampling** as implemented in sklearn's StratifiedKFold and StratifiedShuffleSplit to ensure that relative class frequencies is approximately preserved in each train and validation fold.

More details about different cross-validation strategies, implemented in sklearn, can be found [here](http://scikit-learn.org/stable/modules/cross_validation.html)
### =====================================================

Prepare KFold with 5 splits, stratified by target variable, shuffled, with fixed random_state = 42
<br>**Don't forget to filter by column 'is_train'!**
<br>Fit models on subset of features: [columns_to_use]

In [17]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [18]:
df.filter('is_train')

""
0
1
2
3
4
5
6
7
8
9


In [19]:
X_train = df.iloc[:6499, :]

In [20]:
X_train

,bruises,cap_color,cap_shape,cap_surface,gill_attachment,gill_color,gill_size,gill_spacing,habitat,odor,population,ring_number,ring_type,spore_print_color,stalk_color_above_ring,stalk_color_below_ring,stalk_root,stalk_shape,stalk_surface_above_ring,stalk_surface_below_ring,target,veil_color,is_train
0,0,0,2,2,1,10,1,0,4,7,3,1,4,1,7,7,2,0,3,3,0.0,2,True
1,0,3,3,0,1,1,0,0,6,6,4,1,4,1,7,7,0,1,3,3,1.0,2,True
2,1,0,3,3,0,6,0,0,1,6,4,1,4,1,4,4,3,0,3,3,0.0,1,True
3,0,3,2,0,1,1,0,0,6,6,5,1,4,0,7,7,0,1,3,3,1.0,2,True
4,1,0,4,3,1,2,1,0,3,4,4,1,0,7,5,5,3,1,2,3,0.0,2,True
5,1,0,3,2,1,2,1,0,1,4,4,1,0,7,7,5,3,1,3,3,0.0,2,True
6,1,7,2,3,1,2,1,0,1,3,4,1,0,7,5,5,3,1,3,2,0.0,2,True
7,1,0,3,2,1,2,1,0,3,8,4,1,0,7,5,5,3,1,2,3,0.0,2,True
8,1,3,0,0,1,7,0,1,0,6,2,2,4,7,7,7,3,0,2,3,1.0,2,True
9,0,8,3,3,1,10,0,0,0,4,4,1,4,3,7,7,0,1,3,0,1.0,2,True


In [21]:
y_train = X_train['target']

In [22]:
X_train.drop('target', axis=1, inplace=True)

/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
X_train.drop('is_train', axis=1, inplace=True)

/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
y_train

0       0.0
1       1.0
2       0.0
3       1.0
4       0.0
5       0.0
6       0.0
7       0.0
8       1.0
9       1.0
10      1.0
11      1.0
12      1.0
13      0.0
14      1.0
15      0.0
16      1.0
17      0.0
18      0.0
19      1.0
20      0.0
21      0.0
22      1.0
23      0.0
24      0.0
25      0.0
26      0.0
27      1.0
28      1.0
29      0.0
       ... 
6469    1.0
6470    0.0
6471    1.0
6472    1.0
6473    0.0
6474    1.0
6475    1.0
6476    1.0
6477    1.0
6478    1.0
6479    1.0
6480    0.0
6481    1.0
6482    1.0
6483    1.0
6484    0.0
6485    1.0
6486    0.0
6487    0.0
6488    1.0
6489    1.0
6490    1.0
6491    0.0
6492    1.0
6493    1.0
6494    1.0
6495    0.0
6496    1.0
6497    0.0
6498    0.0
Name: target, Length: 6499, dtype: float64

In [25]:
from os import cpu_count

n_jobs = max(cpu_count()-1, 1)
# your code goes here
# ---------------------------------------------------------------
# cross-validation iterator
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# xtrain, ytrain, DataFrame-like
xtrain = X_train
ytrain = y_train
# ---------------------------------------------------------------

# create Decision Tree with default params, max_depth=4, random_state=42
dt = DecisionTreeClassifier(
    max_depth=3, random_state=42
)

# estimate its f1-score with cross-validation (cross_val_score)
# your code goes here
# ---------------------------------------------------------------
scores_dt = cross_val_score(
    estimator=dt,
    X=xtrain,
    y=ytrain,
    scoring='f1', 
    cv=kf, # cross-validation strategy
    n_jobs=n_jobs
).mean()
print('DT scoring: {:.2f}'.format(scores_dt))
# ---------------------------------------------------------------


# create Logistic Regression with default params, random_state=42
lr = LogisticRegression(
    random_state=42
)

# estimate its f1-score with cross-validation
# your code goes here
# ---------------------------------------------------------------
scores_lr = cross_val_score(
    estimator=lr,
    X=xtrain, # ...
    y=ytrain, # ...
    scoring='f1',
    cv=kf,
    n_jobs=n_jobs
).mean()
print('LR scoring: {:.2f}'.format(scores_lr))

DT scoring: 0.90
LR scoring: 0.89


Why is a score of Linear Regression lower than correspondent one of DT?
1. Is everything OK with the data format for linear models? (revision of 2 previous lectures). 
1. If not, what else you should do to use the data appropriately for linear models?
1. Why didn't point 1. affect Decision Tree performance?

### Task 4. Now it's time to do some hyperparam tuning
Perform suitable hyperparam tuning using created above cross-validation strategy
<br>Main parameters to perform grid-search for:
- max_depth (1,2,...None)
- min_samples_leaf (1,2,...)
- criterion (gini, entropy)
- weight (none, balanced)
- max_features (sqrt(features), 50%, 75%, all of them, ...)
- other params available, see documentation

So - use your fantasy for filling-in abovementioned lists

You should receive **a gain of 0.01 in f1-score or higher**
<br>(current benchmark = +0.0268 gain)

In [26]:
from math import sqrt

In [27]:
n_features = X_train.shape[1]
n_features

21

In [28]:
%%time
# your code goes here
# ---------------------------------------------------------------
# create base model (DT, random state = 42)
estimator = DecisionTreeClassifier(
    random_state=42
)

# create parameter grid
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
params = {
    'max_depth': [6, None],
    'min_samples_leaf': [9],
    'criterion': ('gini', 'entropy'),
    'class_weight': (None, 'balanced'),
    'max_features': [10, 15, 17, 19, 20],
    'presort': (True, False),
    'max_leaf_nodes': [18, None],
}

# create grid search object
gs = GridSearchCV(
    estimator=estimator,  # base model
    param_grid=params,  # params grid to search within
    cv=kf,  # cross-validation strategy
    #error_score=1,  # warnings only
    scoring='f1',  # f1-score
    # thread count, the higher count - the faster
    n_jobs=n_jobs,
    verbose=2,  # messages about performed actions
)

# perform grid search on TRAIN dataset ('is_train' filtering)
gs.fit(
    X=X_train, # ...
    y=y_train, # ...
)
# -------------------------------------------------------------
# extract best score on cross-validation
best_score = gs.best_score_
# extract the estimator (DT) with best params on cross-validation
best_dt = gs.best_estimator_
# check gain in f1-score
print('f1-score best: {:.4f}, +{:.4f} better than baseline'.format(
    best_score, (best_score - scores_dt))
)

Fitting 5 folds for each of 160 candidates, totalling 800 fits
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True 
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True 
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True 
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True, total=   0.0s
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True, total=   0.0s
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True 
[CV] class_weight=None, criteri

[CV] class_weight=None, criterion=gini, max_depth=6, max_features=15, max_leaf_nodes=18, min_samples_leaf=9, presort=False 
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=15, max_leaf_nodes=None, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=15, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=15, max_leaf_nodes=18, min_samples_leaf=9, presort=False, total=   0.0s
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=15, max_leaf_nodes=None, min_samples_leaf=9, presort=False, total=   0.0s
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=15, max_leaf_nodes=None, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=17, max_leaf_nodes=18, min_samples_leaf=9, presort=True 
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=

[CV] class_weight=None, criterion=gini, max_depth=6, max_features=17, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=19, max_leaf_nodes=18, min_samples_leaf=9, presort=False 
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=17, max_leaf_nodes=None, min_samples_leaf=9, presort=False, total=   0.0s
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=19, max_leaf_nodes=18, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=17, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=19, max_leaf_nodes=18, min_samples_leaf=9, presort=False 
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=19, max_leaf_nodes=None, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=19, max_leaf_n

[CV] class_weight=None, criterion=gini, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=True, total=   0.0s
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=False, total=   0.0s
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=20, max_leaf_nodes=18, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=True 
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=20, max_leaf_nodes=18, min_samples_leaf=9, presort=False 
[CV] class_weight=None, criterion=gini, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV]  class_weight=None, criterion=gini, max_depth=6, max_features=20, max_leaf_

[CV] class_weight=None, criterion=gini, max_depth=None, max_features=15, max_leaf_nodes=18, min_samples_leaf=9, presort=False 
[CV]  class_weight=None, criterion=gini, max_depth=None, max_features=15, max_leaf_nodes=18, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=None, max_features=15, max_leaf_nodes=None, min_samples_leaf=9, presort=True 
[CV]  class_weight=None, criterion=gini, max_depth=None, max_features=15, max_leaf_nodes=18, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=None, max_features=15, max_leaf_nodes=None, min_samples_leaf=9, presort=True 
[CV]  class_weight=None, criterion=gini, max_depth=None, max_features=10, max_leaf_nodes=None, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=None, criterion=gini, max_depth=None, max_features=10, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV]  class_weight=None, criterion=gini, max_depth=None, max_

...

[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True 
[CV]  class_weight=balanced, criterion=entropy, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=True 
[CV]  class_weight=balanced, criterion=entropy, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=6, max_features=20, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=10, max_leaf_nodes=18, min_samples_leaf=9, presort=True 
[CV]  class_weight=balance

[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=10, max_leaf_nodes=None, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=10, max_leaf_nodes=None, min_samples_leaf=9, presort=True 
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=15, max_leaf_nodes=18, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=15, max_leaf_nodes=18, min_samples_leaf=9, presort=True 
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=15, max_leaf_nodes=None, min_samples_leaf=9, presort=True, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=15, max_leaf_nodes=None, min_samples_leaf=9, presort=True 
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=15, max_leaf_nodes=18, min_samples_leaf=9, presort=True, total=   0.0s
[

[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=17, max_leaf_nodes=18, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=17, max_leaf_nodes=18, min_samples_leaf=9, presort=False 
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=17, max_leaf_nodes=None, min_samples_leaf=9, presort=False, total=   0.0s
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=17, max_leaf_nodes=18, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=17, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=19, max_leaf_nodes=18, min_samples_leaf=9, presort=False 
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=17, max_leaf_nodes=None, min_samples_leaf=9, presort=False, total=   0

[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=20, max_leaf_nodes=18, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=20, max_leaf_nodes=18, min_samples_leaf=9, presort=False 
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=19, max_leaf_nodes=None, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=19, max_leaf_nodes=None, min_samples_leaf=9, presort=False 
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=19, max_leaf_nodes=None, min_samples_leaf=9, presort=True, total=   0.1s
[CV]  class_weight=balanced, criterion=entropy, max_depth=None, max_features=20, max_leaf_nodes=18, min_samples_leaf=9, presort=False, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, max_depth=None, max_features=20, max_leaf_nodes=18, min_samples_leaf=9, presort=Fals

[Parallel(n_jobs=3)]: Done 800 out of 800 | elapsed:    9.2s finished


In [29]:
# take a look on the best model, compare with the baseline
best_dt

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=15, max_leaf_nodes=18, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=9,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=True, random_state=42, splitter='best')

In [30]:
X_test = df.iloc[6499:, :]

In [31]:
X_test.drop('target', axis=1, inplace=True)
X_test.drop('is_train', axis=1, inplace=True)

/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
# check performance on holdout dataset, unseen before (filter 'is_train' == False)

# your code goes here
# ---------------------------------------------------------------
# appropriate df_test data subset from 'df' dataframe
xtest = X_test
# fit baseline model 'dt' on xtrain, ytrain (because it's not fitted yet)
dt.fit(xtrain, ytrain)
# ---------------------------------------------------------------

# baseline model
y_true = pd.read_csv('/Users/prophet/Documents/GitHub/udscourse/data/4-mushrooms-y_test.csv')
y_pred_baseline = dt.predict(xtest)

print('Base on train:   {:.4f}\nBase on holdout: {:.4f}\ndiff: {:.4f}'.format(
    scores_dt, 
    f1_score(y_true, y_pred_baseline),
    scores_dt - f1_score(y_true, y_pred_baseline)
))

# best model
y_pred_best = best_dt.predict(xtest)

print('\nBest on train:   {:.4f}\nBest on holdout: {:.4f}\ndiff: {:.4f}'.format(
    best_score, 
    f1_score(y_true, y_pred_best),
    best_score - f1_score(y_true, y_pred_best)
))

Base on train:   0.9011
Base on holdout: 0.8888
diff: 0.0124

Best on train:   0.9280
Best on holdout: 0.9205
diff: 0.0075


Now you can see that 
<br>**absolute values of f1-score is higher and distance between train|holdout is lower** <br>for **best model** in comparison to **baseline**

**Bonus question**:

Consider two possibilities:
- (a) you have trained **one best** (on cross-validation) Decision Tree
- (b) you randomly choose 25 subsets of 70% of training data, fits "overfitted" (max_depth=None) Decision Trees on it - each of them performs slightly worse than Tree in (a), and then average predicted results over all 25 models (overfitted trees)

**Which one of them would most likely give the best results on hold-out dataset? What makes you think that way?**